In [55]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *
import qubx.pandaz.ta as pta

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal

from qubx import QubxLogConfig
from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer
from qubx.core.metrics import chart_signals

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Strat for signals testing

In [2]:
class StrategyForTracking(IStrategy):
    timeframe: str = "1Min"
    fast_period = 5
    slow_period = 12

    def on_fit(self, ctx: StrategyContext, fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f" -> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        signals = []
        for i in ctx.instruments:
            # logger.info(f"\t{i.symbol} : {ctx.broker_provider.get_quote(i.symbol)}")
            ohlc = ctx.ohlc(i, self.timeframe)
            fast = sma(ohlc.close, self.fast_period)
            slow = sma(ohlc.close, self.slow_period)
            pos = ctx.positions[i.symbol].quantity

            if pos <= 0:
                if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                    # ctx.trade(i, abs(pos) + i.min_size * 10)
                    signals.append(i.signal(+1, stop=ohlc[1].low))

            if pos >= 0:
                if (fast[0] < slow[0]) and (fast[1] > slow[1]):
                    # ctx.trade(i, -pos - i.min_size * 10)
                    signals.append(i.signal(-1, stop=ohlc[1].high))

        return signals

    def ohlcs(self, timeframe: str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def tracker(self, ctx: StrategyContext) -> PositionsTracker:
        return PositionsTracker(FixedRiskSizer(1, 10_000, reinvest_profit=True))

In [ ]:
r = CsvStorageDataReader("../tests/data/csv")

QubxLogConfig.set_log_level("DEBUG")

rep = simulate({
        "As Strategy": StrategyForTracking(timeframe="15Min", fast_period=5, slow_period=25),
    },
    r, 10000, ["BINANCE.UM:BTCUSDT"], 
    dict(type="ohlc", timeframe="15Min", nback=0),
    "-1Sec",
    "vip0_usdt", 
    "2024-01-01", "2024-01-10", 
#     n_jobs=1
)

In [7]:
tearsheet(rep, False)

In [ ]:
rep[0].executions_log

# Atr Tracker

In [ ]:
from qubx.trackers.riskctrl import AtrRiskTracker
QubxLogConfig.set_log_level("DEBUG")


r = MultiQdbConnector('xlydian-data')
rep = simulate(
    {
        "test0": [pd.Series(
            {
                "2024-01-01 10:00": 1,
                # "2024-01-02 12:00": 0,
                "2024-01-02 12:00": 1,
                "2024-01-03 10:00": -1,
                "2024-01-04 15:00": 0,
            }, name='BTCUSDT'), 
                  AtrRiskTracker(5, 5, '5Min', 32)
            ],
    },

    r, 10000, ["BINANCE.UM:BTCUSDT"], 
    dict(type="ohlc", timeframe="5Min", nback=30), "-1Sec", "vip0_usdt", "2024-01-01", "2024-01-05", 
)

2024-08-02 00:51:51.453 [ 🐞 ] Connected to QuestDB at xlydian-data:8812
2024-08-02 00:51:51.457 [ 🐞 ] 2024-01-01 00:00:00 Initiating simulated trading for binance.um for 10000 x 1.0 in USDT...
2024-08-02 00:51:51.457 [ ℹ️ ] SimulatedData.binance.um initialized


Simulating:   0%|          | 0/313200.0 [00:00<?, ?s/s]

2024-08-02 00:51:51.457 [ 🐞 ] Using pre-generated signals:
 
2024-08-02 00:51:51.457 [ 🐞 ] Triggering strategy on every 5Min bar after 
2024-08-02 00:51:51.457 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '5Min', 'nback': 30} for 
	['BTCUSDT'] 
2024-08-02 00:51:51.457 [ 🐞 ] SimulatedExchangeService :: subscribe :: BTCUSDT :: ohlc
2024-08-02 00:51:51.457 [ ℹ️ ] SimulatedExchangeService :: run :: Simulation started at 2024-01-01
2024-08-02 00:51:51.474 [ ℹ️ ] Initializing chunks for each loader
2024-08-02 00:51:51.589 [ 🐞 ] [2023-12-31T21:30:00.001000000]: Invoking _GeneratedSignalsStrategy on_fit('2024-01-01 00:00:00', 'None')
2024-08-02 00:51:51.589 [ 🐞 ] [2023-12-31T21:30:00.001000000]: _GeneratedSignalsStrategy is fitted
2024-08-02 00:51:51.656 [ 🐞 ] (StrategyContext) sending market buy for 1.0 of BTCUSDT ...
2024-08-02 00:51:51.656 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100001 MARKET BUY 1.0 executed at 42688.850000000006
2024-08-02 00:51:51.656 [ 🐞 ]   

2024-08-02 00:51:52.071 [ 🐞 ]   ::  traded 1.0 for BTCUSDT @ 44928.55 -> 449.95
2024-08-02 00:51:52.071 [ 🐞 ] Order SIM-ORDER-BTCUSDT-100006 MARKET BUY 1.0 of BTCUSDT -> CLOSED
2024-08-02 00:51:52.071 [ 🐞 ] [SIM-ORDER-BTCUSDT-100006 / _GeneratedSignalsStrategy_BTCUSDT_17040672006] : MARKET BUY 1.0 of BTCUSDT  -> [CLOSED]
2024-08-02 00:51:52.071 [ 🐞 ] Executed 1.0 @ 44928.55 of BTCUSDT for order SIM-ORDER-BTCUSDT-100006
2024-08-02 00:51:52.071 [ 🐞 ] BTCUSDT >>> Adjusting position from -1.0 to 0 : [SIM-ORDER-BTCUSDT-100006] MARKET BUY 1.0 of BTCUSDT  (gtc) [CLOSED]
2024-08-02 00:51:52.255 [ ⚠️ ] qubx.gathering.simplest:alter_position_size:17 - Can't change position size for BINANCE.UM:BTCUSDT [CRYPTO PERPETUAL (1.0) 2019-09-25T11:00:00 -> 2100-12-25T10:00:00]. Current position: 0.0, requested size: 0.0
2024-08-02 00:51:52.255 [ ℹ️ ] SimulatedExchangeService :: run :: Simulation finished at 2024-01-04 15:00:00


In [ ]:
tearsheet(rep, False)

In [62]:
ohlc_pd = r.read('BINANCE.UM:BTCUSDT', '2024-01-01', '2024-02-05', AsPandasFrame())
ohlc_pd = ohlc_resample(ohlc_pd, '5Min')

2024-08-02 00:50:28.222 [ 🐞 ] Closing connection
2024-08-02 00:50:28.222 [ 🐞 ] Closing connection
2024-08-02 00:50:28.222 [ 🐞 ] Closing connection
2024-08-02 00:50:28.229 [ 🐞 ] Closing connection


In [69]:
chart_signals(rep[0], "BTCUSDT", ohlc_pd, overlay=[
    ohlc_pd.close - 5*pta.atr(ohlc_pd, 32),
    ohlc_pd.close + 5*pta.atr(ohlc_pd, 32),
])

(None, None)